# Portfolio Experiment

## Working Environment

#### Please ensure that you have the following packages installed
math; pandas; numpy; matplotlib; random; scipy; hmmlearn; sklearn

In [1]:
from main_head import *

# Experiment 1: Empirical Studies Section 4.3 - 4.5
## Overall Parameter Setup
The overall parameter choice is set in the ``CVaR_parameter.py``. Below are some of parameters selected in our experiment.

In [2]:
# epsilon is the fixed parameter regarding optimizing CVaR, representing 1-\eta
epsilon = 0.05

# for the whole rolling approach, the number of days we consider in each period
## It is not just the physical days. Instead, in our paper, we consider the month as a unit. 
rolling_day = 1

# It denotes whether we include the shortsale constraints in our optimization model
## 0 represents that we incorporate the shortsale constraint: x_i >= 0 in our paper
shortsale_sign = 0

# cross validation choice
## cv_type == -1 means no  
## cv_type == 1 means gridsearch
cv_type = 1

# the K-fold number in cross validation
fold_number = 4

# the possible theta param corresponding to m in the paper) used in the cross validation
theta_param = [0.02, 0.04, 0.06, 0.08, 0.1]

# whether we take the risk-free rate into account in computing SR
## true means we need to minus the risk-free rate from the dataset and false means not
## Because DeMiguel's paper does not incorporate risk free rate, we do not incorporate that item too in the test statistics computation.
sharpe_ratio_open = False

#to make it more clearly, unit = 1 means the unit of portfolio dta is x%, unit = 100 means the unit of portfolio dta is x. 
unit = 1

## Initial Specific Data Set Choice
Below are the parameters that fix the data sets in the $\texttt{factor model}$ folder. We choose the concrete dataset in that folder. By changing those parameters, we can switch to different datasets.

In [3]:
#choose the dataset name and the file path
portfolio_number = 16
freq = "Monthly"
#Here as we all use the Monthly Data Set, we can fix that parameter

value = ""
#eq/not eq csv in the . Here we all use the original dataset class.

#dta type
## In DeMiguel's paper, the dta type are selected to be one in ['MKT', 'Country', 'S&P_sectors', 'IndustryPortfolios', 'FF']
## When his dataset extended to 2019 Q1, the dta type are selected to be one in ['MKT2', 'Country2', 'S&P_sectors2', 'IndustryPortfolios2', 'FF22']
## Their corresponding portfolio number is 3, 9, 11, 11, 21(24)
## And in our case study, dta type are selected to be 'IndustryIndices' (16)
data_type = "IndustryIndices"

#select part of the data in the .csv file.
## from start_time to train_test_split as the default train dataset; from train_test_split (not include) as the test dataset.
start_time = 20040101
end_time = 201905
train_test_split = 20061231

## Load the Data
we load the return data and its corresponding factor data below using the class from ``data_process/input.py``.

Note that the expressions for the model are explicitly written in the paper. We apply a rolling-sample approach with window size to be $M$. For the overall $T$ monthly-long data, in each month $t$, we use the dataset from months $t$ to $t + M - 1$ as input to solve the portfolio optimization problem. Then, we obtain the portfolio weights by solving the optimization problem for month $t + M$, where optimized weights $x_{i}^*$ are used to compute the returns $t + M$.

In [4]:
#data input
Input_csv = Input(portfolio_number, freq, value, start_time, end_time, train_test_split, data_type)
[data_head, data_parameter, csv_name] = Input_csv.parameter_output()
[df_select, df_train] = Input_csv.data_load()

df_factor = Input_csv.three_factor_load()
#we do not include the risk free rate item into the computation of sharpe ratio
if sharpe_ratio_open == False:
    rfr_data = 0
else:
    rfr_data = Input_csv.risk_free_rate()

## Evaluation 
To evaluate the performance of each model, we compute the following return statistics similar to what have done in DeMiguel et al. (2009b).The return statistics are computed in ``data_process/test_result.py`` and the turnover metric is computed in each model. We refer the output information to the ``result_hmm/xx.csv``.
## Output Return Statistics Computation & Comparison
### Sharpe ratio
We denote the out-of-sample Sharpe ratio as the sample mean of the out-of-sample excess return $\hat{\mu}$ over the risk-free asset, then divided by their sample standard deviation, $\hat{\sigma}$:
$$\text{Sharpe ratio} = \frac{\hat{\mu}}{\hat{\sigma}}$$
### CEQ
We measure the certainty equivalent (CEQ) return, defined as the risk-free rate that an investor is willing to take compared to a specific model.
$$\text{CEQ} = \hat{\mu} - \frac{1}{2}\hat{\sigma}^2.$$
### drawdown
(Maximum) drawdown can be extracted from the formula: $-\min\{\boldsymbol r_{M + 1},\boldsymbol r_{M + 2},...,\boldsymbol r_T \}.$
### turnover
This is computed by the average of the $\ell_1$ norm of the trades across the $N$ risky assets:
$$\text{Turnover} = \frac{1}{T - M}\sum_{t = 1}^{T - M}\|\boldsymbol x_{t + 1}^* - \boldsymbol x_{t_+}\|_1,$$
where $\boldsymbol x_{t+1}^*$ is the optimal portfolio weight vector at time $t + 1$ and $\boldsymbol x_{t_+}$ is th portfolio weight right before rebalancing at time $t + 1$.

In [5]:
#initiation for the test metric module
#print the output into a separate csv file, separated by the dataset type + header
Output_csv = output(csv_name, data_head, data_parameter)
Output_csv.head()

done


## Benchmark

## Benchmark Strategies
$\texttt{method.rolling(shortsale)}$ represents the rolling-sample approach for each model.
### Equally Weighted Portfolio
This equally weighted strategy means that we consider the portfolio weight to be as follows for each of the $I$ risky assets:
$$w_i = \frac{1}{I},$$
which gains popularity for its robustness in DeMiguel et al. (2009b).

In [12]:
# EW Portfolio
naive = naive_strategy(df_select, df_train, rolling_day, portfolio_number, "EW portfolio")
naive.rolling()
Output_csv.return_info(naive, rfr_data)

184 36
For the model with strategy name  EW portfolio
The Monthly Sharpe Ratio =  0.10897235636499307
CEQ =  0.0029030162860526417
Drawdown =  -0.3829684339776208
Turnover =  0.037593675961054306


The following model requires the information of history returns $\{\boldsymbol{r}_1, \boldsymbol{r}_2,...,\boldsymbol{r}_M\}$.
## CVaR
The optimization problem for the $\texttt{CVaR}$ metric is computed below:
$$\min_{\boldsymbol x \in \mathcal{X}, v \in \mathbb{R}}\left\{v+\frac{1}{1-\eta} \frac{1}{M}\sum\limits_{i \in[M]}\left(-\boldsymbol r_{i}^{\prime} \boldsymbol x-v\right)^{+}\right\}$$

In [13]:
# SAA-CVaR model
saa_CVaR = SAA_CVaR(df_select, df_train, rolling_day, portfolio_number, 'CVaR')
train_return = saa_CVaR.rolling()  
Output_csv.return_info(saa_CVaR, rfr_data)

Using license file D:\ProgramFiles\gurobi903\gurobi.lic
Academic license - for non-commercial use only
For the model with strategy name  CVaR
The Monthly Sharpe Ratio =  0.013843392144446654
CEQ =  6.174958883421348e-05
Drawdown =  -0.26944224262581784
Turnover =  0.16242807201509424


## DR CVaR (Wasserstein)
Common ways to illustrate the Wasserstein ambiguity set:
$$F_{W}(\theta)=\left\{\mathbb{P} \in \mathcal{P}\left(\mathbb{R}^{I}\right) ~\left\vert~ \begin{array}{ll}{\tilde{\boldsymbol r} \sim \mathbb{P}}\\ {\hat{\mathbb{P}}_M:=\frac{1}{M}\sum_{n = 1}^{M}\delta_{\boldsymbol r_{n}}}\\{P(W_p(\mathbb{P},\hat{\mathbb{P}}_M) \leq \theta) = 1}\end{array}\right.\right\}.$$

In [14]:
method_name = "DR CVaR (Wasserstein)"
mean_constr = False
fcvar_wasserstein2 = FCVaR_wasserstein2(df_select, df_train, rolling_day, portfolio_number, 1, method_name, cv_type, mean_constr)
fcvar_wasserstein2.rolling()
Output_csv.return_info(fcvar_wasserstein2, rfr_data)

For the model with strategy name  DR CVaR (Wasserstein)
The Monthly Sharpe Ratio =  0.10809414899889463
CEQ =  0.002631628265429075
Drawdown =  -0.31969271666246124
Turnover =  0.1924298093644551


# DR Mean-CVaR (Momemts)
The method proposed in ``Kang et al. (2019)``


In [16]:
meancvar = mean_FCVaR(df_select, df_train, rolling_day, portfolio_number, 'DR Mean-CVaR')
meancvar.rolling()
Output_csv.return_info(meancvar, rfr_data)

0 2.629752227
1 2.58305527
2 2.58305527
3 2.588730249
4 2.301049666
5 2.301049666
6 2.304304127
7 2.26754787
8 2.3023316649999996
9 2.265631665
10 2.268374181
11 2.4148627510000003
12 2.374762751
13 2.496579917
14 2.453389515
15 2.4801338370000003
16 2.48054584
17 2.484271741
18 2.49421032
19 2.530902651
20 2.682711831
21 2.801482001
22 4.378864001
23 4.498623681
24 4.625003276999999
25 4.817703277
26 4.808049565999999
27 4.8037495660000005
28 5.498441236
29 5.736541236000001
30 5.736541236000001
31 5.736541236000001
32 5.736541236000001
33 5.736541236000001
34 5.736541236000001
35 5.737241236
36 5.742690493
37 5.742690493
38 5.742690493
39 5.742690493
40 5.736363921
41 5.736363921
42 5.736363921
43 5.736363921
44 5.736363921
45 5.736363921
46 5.736363921
47 5.736363921
48 5.736363921
49 5.736363921
50 5.736363921
51 5.736363921
52 5.749106288
53 5.753233632000001
54 5.753233632000001
55 5.753233632000001
56 5.731433632
57 5.744793614
58 4.657131919
59 4.570397891
60 4.36191551
61 4.27

In [19]:
mvp = MVP(df_select, df_train, rolling_day, portfolio_number, df_factor, 'MVP')
mvp.rolling(0)
Output_csv.return_info(mvp, rfr_data)

For the model with strategy name  MVP
The Monthly Sharpe Ratio =  0.029734779023708795
CEQ =  0.0003172302476354917
Drawdown =  -0.20196165266650556
Turnover =  0.09954737933550041


## Incoporate the Mean Constraint
### How to incorporate the mean constraint:
For a traditionally problem, the mean constraint refers to: $\boldsymbol \mu^{\prime}\boldsymbol x \geq R.$ 

In the distributionally robust version, we incorporate the following mean constraint:
$$\inf_{\mathbb{P} \in F} \mathbb{E}_{\mathbb{P}}[\boldsymbol r^{\prime}\boldsymbol x]\geq R.$$

## Our Proposed Model

In [18]:
#whether to incorporate the worst-case mean constraint
mean_sign = 'Mean-'

#choice_index = 0 --> Bull & Bear;
#choice_index = 1 --> Weathers
#choice_index = 2 --> HMM
choice_index = 2
hmm_type = -1

if mean_sign == "":
    mean_constr = False
else:
    mean_constr = 'worst-case'

cls_num_choice = [2, 4, 4]
state_choice = ['BB', 'weathers', 'HMM']
method_choice = ['(Bull & Bear)', '(Weathers)', '(HMM)']    

### Pretrained for determining the Markovian States
We use the MATLAB hmmtrain function for training the HMM model.

And we use the difference between predicted GDP (CPI) and true GDP (CPI)to classify into 2 states.

And Bull & Bear state results are saved in ``factor model/BB_state.csv``;

Weathers results are saved in ``factor model/weathers_state.csv``;

HMM results are saved in ``factor model/HMM_state.csv``.



### Regime Determination
We would first load the HMM / Weathers / Bull & Bear Classification result within the right train test time.

In [19]:
# Pretrain for Heterogeneous I
cluster_num = cls_num_choice[choice_index]
df_state_case = pd.read_csv('../factor model/' + state_choice[choice_index] + '_state.csv')
str_state = [str(each_state) for each_state in list(df_state_case['Date'])]
df_state_case['Date'] = str_state
mkt_state = df_state_case[((df_state_case['Date'])>=str(start_time))&(df_state_case['Date']<str(end_time))]['state']


#### RS CVaR Model
i.e. our proposed model with $\theta = 0$.

In [20]:
method_name = 'RS ' + mean_sign + 'CVaR ' + method_choice[choice_index]
cv_type = -1
mean_cvar_mkt = FCVaR_HMM_wasserstein(df_select, df_train, rolling_day, portfolio_number, df_factor, cluster_num, method_name, mkt_state, hmm_type, cv_type, mean_constr)
mean_cvar_mkt.theta = 0
mean_cvar_mkt.rolling()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147


In [21]:
Output_csv.return_info(mean_cvar_mkt, rfr_data)

For the model with strategy name  RS Mean-CVaR (HMM)
The Monthly Sharpe Ratio =  0.08175997523167387
CEQ =  0.0016319633090451178
Drawdown =  -0.2714302341705067
Turnover =  0.5865672746850097


#### Our RSDR model

In [22]:
# RSDR CVaR (HMM)
cv_type = 1
method_name = 'RSDR ' + mean_sign + 'CVaR ' + method_choice[choice_index]
mean_fcvar_mkt = FCVaR_HMM_wasserstein(df_select, df_train, rolling_day, portfolio_number, df_factor, cluster_num, method_name, mkt_state, hmm_type, cv_type, mean_constr)
mean_fcvar_mkt.rolling()

0
1
Unable to retrieve attribute 'objVal'
2
Unable to retrieve attribute 'objVal'
Unable to retrieve attribute 'objVal'
3
Unable to retrieve attribute 'objVal'
4
Unable to retrieve attribute 'objVal'
Unable to retrieve attribute 'objVal'
5
Unable to retrieve attribute 'objVal'
Unable to retrieve attribute 'objVal'
6
Unable to retrieve attribute 'objVal'
Unable to retrieve attribute 'objVal'
7
Unable to retrieve attribute 'objVal'
8
Unable to retrieve attribute 'objVal'
Unable to retrieve attribute 'objVal'
9
Unable to retrieve attribute 'objVal'
Unable to retrieve attribute 'objVal'
10
Unable to retrieve attribute 'objVal'
11
Unable to retrieve attribute 'objVal'
Unable to retrieve attribute 'objVal'
12
Unable to retrieve attribute 'objVal'
Unable to retrieve attribute 'objVal'
13
14
15
Unable to retrieve attribute 'objVal'
16
17
18
19
Unable to retrieve attribute 'objVal'
20
21
Unable to retrieve attribute 'objVal'
Unable to retrieve attribute 'objVal'
22
Unable to retrieve attribute 

In [23]:
#return statistics computation for our policy
Output_csv.return_info(mean_fcvar_mkt, rfr_data)

For the model with strategy name  RSDR Mean-CVaR (HMM)
The Monthly Sharpe Ratio =  0.13627582466731109
CEQ =  0.0034332550342641138
Drawdown =  -0.3251899604822308
Turnover =  0.24899216262419097
